# Import Libraries
  

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import sys
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from google.colab import drive
from keras.models import load_model
import yaml
import glob
from keras.models import Model
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Get Data
Data is obtained by concatenating all the csv files obtained from simulation using PID controllers. The collect_data.py script is used to generate these csv files during simulation


In [0]:
def get_data():
    PATH = '/content/drive/My Drive/Neural Network Controller/data/'
    files = glob.glob(f'{PATH}*.csv')[:]
    print(files)
    df = pd.concat([pd.read_csv(f) for f in files])
    print(df.head())
    df = df.drop(['id'], axis = 1)
    X = df.drop(['target'], axis = 1)
    Y = pd.DataFrame(df['target'])
    return X.values, Y.values


In [0]:
def split_data(X, Y):
#     X, Y = shuffle(X, Y)
    length = X.shape[0]
    split = int(length * 0.9)
    X_train = X[:split]
    Y_train = Y[:split]
    X_test = X[split:]
    Y_test = Y[split:]

    return X_train, Y_train, X_test, Y_test

In [0]:
X, Y = get_data()
X_train, Y_train, X_test, Y_test = split_data(X, Y)
print(X_train.shape)

['/content/drive/My Drive/Neural Network Controller/data/data1.csv', '/content/drive/My Drive/Neural Network Controller/data/data2.csv', '/content/drive/My Drive/Neural Network Controller/data/data3.csv']
   id       cur_pos       cur_vel         error  goal_pos        target
0   0 -4.882628e-10 -2.338387e-09  4.882628e-10       0.0  9.813957e-06
1   1  4.883240e-10  2.454111e-09 -4.883240e-10       0.0 -9.815358e-06
2   2 -4.834622e-10  1.027551e-08  4.834622e-10       0.0  5.470557e-08
3   3  4.877077e-10  3.308846e-09 -4.877077e-10       0.0 -9.801325e-06
4   4  4.882628e-10  2.537998e-09 -4.882628e-10       0.0 -9.813948e-06
(25678, 4)


# Build and Compile Model

The model is 4 layers with relu activation function for each layer except the last one. Since the task is regression, the final layer activation function is linear. 

The mean absolute error is used as loss as mean squared error tends to give undesirable weightage to outliers

In [0]:
def get_model():
  model = Sequential()
  model.add(Dense(128, kernel_initializer = 'normal', activation = 'relu', input_shape = (4,)))
  model.add(Dense(256, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dense(256, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
  model.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mse', 'mae'])
  model.summary()
  return model

In [0]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 99,713
Trainable params: 99,713
Non-trainable params: 0
_________________________________________________________________


# Train Model
 
 Model is trained for 200 epochs with batch size of 16. After training, the model is saved to be retrieved later to extract the weights and store them in a format that can be easily read in C++

In [0]:
model.fit(X_train, Y_train, batch_size = 16, epochs = 200)

In [0]:
model.evaluate(X_test, Y_test)
model.save('/content/drive/My Drive/Neural Network Controller/models tensorflow/model_2.h5')

2854/2854 [==============================] - 0s 72us/step


In [0]:
model = load_model('/content/drive/My Drive/Neural Network Controller/models tensorflow/model_2.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 99,713
Trainable params: 99,713
Non-trainable params: 0
_________________________________________________________________


# Extract trained model weights to file

Since the weights are stored as h5 files, reading them directly in C++ is nontrivial. To simplify this I iterate over the layers in the trained model, extract the weights, biases and activation functions and store them in a text file that can be easily processed by the controller plugin

In [0]:
with open('/content/drive/My Drive/Neural Network Controller/models tensorflow/weights_3.txt', 'w') as file:
  params = {}
  file.write(str(len(model.layers)))
  file.write('\n')
  for i, layer in enumerate(model.layers):
    weights = layer.get_weights()
    config = layer.get_config()
    weights[1] = weights[1].reshape((1, weights[1].shape[0]))
    params['W' + str(i + 1)] = weights[0]
    params['b' + str(i + 1)] = weights[1]
    file.write('W' + str(i + 1))
    file.write('\n')
    file.write(' '.join('%s' % x for x in weights[0].shape))
    file.write('\n')
    file.write(config['activation'])
    file.write('\n')
    weights[0].tofile(file, sep=" ", format="%s")
    print(weights[0].shape)
    file.write('\n')
    file.write('b' + str(i + 1))
    file.write('\n')
    file.write(' '.join('%s' % x for x in weights[1].shape))
    file.write('\n')
    weights[1].tofile(file, sep=" ", format="%s")
    file.write('\n')
    print(weights[1].shape)
    


(4, 128)
(1, 128)
(128, 256)
(1, 256)
(256, 256)
(1, 256)
(256, 1)
(1, 1)


In [0]:
print(params['W1'])

In [0]:
layer_name = 'dense_11'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x)
print(intermediate_output)
